In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys,os
sys.path.append('./sailfish/')


# Niestabilność Helmholtza


https://www.youtube.com/watch?v=ELaZ2x42dkU&hd=1

In [ ]:
%%writefile ex4.py

import numpy as np
from sailfish.subdomain import Subdomain2D
from sailfish.node_type import NTFullBBWall, NTEquilibriumVelocity
from sailfish.controller import LBSimulationController
from sailfish.lb_single import LBFluidSim

class MyBlock(Subdomain2D):
    max_v = 0.1

    def boundary_conditions(self, hx, hy):
        wall_mapv1 = ( (hy == self.gy-1) )
        wall_mapv0 = ( (hy == 0) )
        
        self.set_node(  wall_mapv1 , NTEquilibriumVelocity((self.max_v, 0.0)) )
        self.set_node(wall_mapv0, NTFullBBWall)
        
    def initial_conditions(self, sim, hx, hy):
        nx,ny = self.gx,self.gy
        sim.rho[:] = 1.0
        sim.vy[(hx==2)&(hy==ny/2-1)] = -0.01
        #border = (hy>2*np.cos(hx/float(nx)*2*np.pi)+ny/2)
        border = (hy>ny/2)       
        #border = (hy>ny/2) | ((hy==ny/2) & (hx>nx/3.) & (hx<2*nx/3.))

        sim.vx[border] = self.max_v
        
        
class MySim(LBFluidSim):
    subdomain = MyBlock

LBSimulationController(MySim).run()

In [ ]:
%%sh
export PYTHONPATH=$PWD/sailfish/
python ex4.py --max_iters=10000 --every=100 \
 --lat_nx=1022 --lat_ny=254 \
 --periodic_x --model=mrt \
 --visc=0.001 \
 --output=ex4A --output_format=npy



In [ ]:
!ls -ltra ex4*.npz|head

In [ ]:
data = np.load("ex4A.0.01000.npz")
vx,vy = data['v']
ny,nx = vx.shape
Y,X = np.mgrid[0:ny,0:nx]
plt.figure(figsize=(12,float(ny)/nx*12))
plt.xlim(1,nx-2)
plt.ylim(1,ny-2)
plt.contourf(X,Y,np.sqrt(vy**2+vx**2),256)
plt.streamplot(X,Y,vx,vy,color='white')


In [ ]:
plt.plot(vx[:,800])

### Plotting a part  the domain

In [ ]:
data = np.load("ex4A.0.04000.npz")
vx,vy = data['v']
s = (slice(80,170,1),slice(20,220,1))

vx, vy  = vx[s],vy[s]
Dxvx,Dyvx = np.gradient(vx)
Dxvy,Dyvy = np.gradient(vy)
ny,nx = vx.shape
Y,X = np.mgrid[0:ny,0:nx]
plt.figure(figsize=(16,float(ny)/nx*16))
plt.xlim(1,nx-2)
plt.ylim(1,ny-2)
plt.contourf(X,Y,Dyvx-Dxvy,256)
plt.streamplot(X,Y,vx,vy,color='white')


### interactive plotting

In [ ]:
import glob
glob.glob("ex4A*.npz")

In [ ]:
import glob
frames = []
s = (slice(80,170,1),slice(20,220,1))

for fn in sorted(glob.glob("ex4A*.npz")):
    vx,vy = np.load(fn)['v']
    vx, vy  = vx[s],vy[s]
    Dxvx,Dyvx = np.gradient(vx)
    Dxvy,Dyvy = np.gradient(vy)
    frames.append([vx, vy,Dyvx-Dxvy,Dxvx])


In [ ]:
print len(frames)


In [ ]:
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets
from IPython.display import display,clear_output
ny,nx = frames[0][0].shape
Y,X = np.mgrid[0:ny,0:nx]

@interact(ith=widgets.IntSlider(min=0,max=len(frames)-1),figsize=widgets.IntSlider(min=0,max=20,value=6))
def g(ith,figsize):
  
    plt.figure(figsize=(figsize,float(ny)/nx*figsize))

    vx,vy=frames[ith][:2]
    plt.xlim(1,nx-2)
    plt.ylim(1,ny-2)
    #plt.contourf(X,Y,frames[ith][2],levels=np.linspace(-0.002,0.002,256))
    plt.imshow(frames[ith][0],vmax=0.1,vmin=-0.1,origin='top')
    #plt.streamplot(X,Y,vx,vy,color='white')
    
    plt.show()

In [ ]:
plt.plot( [np.nanmax(f[0]) for f in frames] )

## Geometry experimentation

In [ ]:
x0,y0,a = 15,8,4
nx,ny=32,32
hy,hx = np.mgrid[0:ny,0:nx]
plt.figure(figsize=(10,float(ny)/nx*10))
# plt.imshow( (hy>2*np.sin(2*hx/float(nx)*2*np.pi)+ny/2),interpolation='nearest') 
#plt.imshow( (hy>ny/2) | ((hy==ny/2) & (hx>nx/3.) & (hx<2*nx/3.) ),interpolation='nearest') 
plt.imshow( hy>5*np.sign(np.sin(2*hx/float(nx)*2*np.pi))+ny/2,interpolation='nearest') 

In [ ]:
np.sign(-.2)